In [1]:
import torch
import utils
import os
import numpy as np
from collections import OrderedDict
from vgg_face_dag import vgg_face_dag

SKIP_LAYERS = utils.SKIP_LAYERS

device = torch.device('cuda')
vgg_weight = './vgg_face_dag.pth'

# ami model
attribute_model = vgg_face_dag(vgg_weight)
attribute_model.to(device)
attribute_model.eval()
attribute_model.set_ami_params(15., 2.15, 15.)
attribute_model.register_my_hook(skip_layers=SKIP_LAYERS, ami_data=utils.load_neuron_set_lists())

# original model
vgg_net = vgg_face_dag(vgg_weight)
vgg_net.to(device)
vgg_net.eval()

register hook for conv1_1
register hook for conv1_2
register hook for pool1
register hook for conv2_1
register hook for conv2_2
register hook for pool2
register hook for conv3_1
register hook for conv3_2
register hook for conv3_3
register hook for pool3
register hook for conv4_1
register hook for conv4_2
register hook for conv4_3
register hook for pool4
register hook for conv5_1
register hook for conv5_2
register hook for conv5_3
register hook for pool5
register hook for fc6
register hook for fc7


Vgg_face_dag(
  (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_1): ReLU(inplace=True)
  (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace=True)
  (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace=True)
  (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace=True)
  (conv3_3): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace=True)
  (pool3)

In [2]:
vgg_root   = '../../data/vgg_face_caffe/'
vgg_names  = utils.read_list(vgg_root + 'names.txt')

In [3]:
img_dir = '/tmp/sized_full_images/'
img_list = utils.read_list('./benign_list.txt')

img_count = 0
adv_count = 0

with torch.no_grad():
    for img_name in img_list:
        img_name  = img_name.strip()
        img_path  = os.path.join(img_dir, img_name)

        prob_original  = vgg_net(utils.get_data(img_path).to(device))
        prob_attribute = attribute_model(utils.get_data(img_path).to(device))
        id_original    = torch.max(prob_original, 1)[1].item()
        id_attribute   = torch.max(prob_attribute, 1)[1].item()

        id_gold = utils.get_identity(img_name, vgg_names)
        
        if id_gold == id_original:
            img_count += 1
        
            ### consistency observer for detecting adversarial samples ###
            if id_original != id_attribute:
                adv_count += 1

            print('{:3} Gold-Original-Attribute: {:4}-{:4}-{:4}  |  Adversary: {:.4f} [{:3}/{:3}]'
                  .format(img_count, id_gold, id_original, id_attribute, 
                          1.0*adv_count/img_count, adv_count, img_count))

  1 Gold-Original-Attribute: 2019-2019-2019  |  Adversary: 0.0000 [  0/  1]
  2 Gold-Original-Attribute: 1992-1992-2066  |  Adversary: 0.5000 [  1/  2]
  3 Gold-Original-Attribute:  451- 451- 451  |  Adversary: 0.3333 [  1/  3]
  4 Gold-Original-Attribute: 2370-2370-2370  |  Adversary: 0.2500 [  1/  4]
  5 Gold-Original-Attribute:  134- 134- 134  |  Adversary: 0.2000 [  1/  5]
  6 Gold-Original-Attribute:  213- 213- 213  |  Adversary: 0.1667 [  1/  6]
  7 Gold-Original-Attribute: 2581-2581-2581  |  Adversary: 0.1429 [  1/  7]
  8 Gold-Original-Attribute:  892- 892- 892  |  Adversary: 0.1250 [  1/  8]
  9 Gold-Original-Attribute: 1754-1754-1754  |  Adversary: 0.1111 [  1/  9]
 10 Gold-Original-Attribute:  586- 586- 586  |  Adversary: 0.1000 [  1/ 10]
 11 Gold-Original-Attribute: 1600-1600- 966  |  Adversary: 0.1818 [  2/ 11]
 12 Gold-Original-Attribute: 2455-2455-2455  |  Adversary: 0.1667 [  2/ 12]
 13 Gold-Original-Attribute: 2051-2051-2051  |  Adversary: 0.1538 [  2/ 13]
 14 Gold-Ori

110 Gold-Original-Attribute: 1433-1433-  93  |  Adversary: 0.1636 [ 18/110]
111 Gold-Original-Attribute: 2489-2489-2489  |  Adversary: 0.1622 [ 18/111]
112 Gold-Original-Attribute: 2179-2179-2179  |  Adversary: 0.1607 [ 18/112]
113 Gold-Original-Attribute:  730- 730- 730  |  Adversary: 0.1593 [ 18/113]
114 Gold-Original-Attribute:  930- 930- 930  |  Adversary: 0.1579 [ 18/114]
115 Gold-Original-Attribute: 2092-2092-2092  |  Adversary: 0.1565 [ 18/115]
116 Gold-Original-Attribute: 1129-1129-1129  |  Adversary: 0.1552 [ 18/116]
117 Gold-Original-Attribute:  442- 442- 442  |  Adversary: 0.1538 [ 18/117]
118 Gold-Original-Attribute:  727- 727- 727  |  Adversary: 0.1525 [ 18/118]
119 Gold-Original-Attribute: 1230-1230-1230  |  Adversary: 0.1513 [ 18/119]
120 Gold-Original-Attribute: 1797-1797-1797  |  Adversary: 0.1500 [ 18/120]
121 Gold-Original-Attribute: 1348-1348-1348  |  Adversary: 0.1488 [ 18/121]
122 Gold-Original-Attribute:  818- 818- 818  |  Adversary: 0.1475 [ 18/122]
123 Gold-Ori

224 Gold-Original-Attribute:  871- 871- 871  |  Adversary: 0.0848 [ 19/224]
225 Gold-Original-Attribute:  401- 401-1087  |  Adversary: 0.0889 [ 20/225]
226 Gold-Original-Attribute:  862- 862- 862  |  Adversary: 0.0885 [ 20/226]
227 Gold-Original-Attribute: 1729-1729-1086  |  Adversary: 0.0925 [ 21/227]
228 Gold-Original-Attribute:  510- 510- 510  |  Adversary: 0.0921 [ 21/228]
229 Gold-Original-Attribute:  177- 177-2158  |  Adversary: 0.0961 [ 22/229]
230 Gold-Original-Attribute: 2112-2112-2112  |  Adversary: 0.0957 [ 22/230]
231 Gold-Original-Attribute:  781- 781- 781  |  Adversary: 0.0952 [ 22/231]
232 Gold-Original-Attribute: 2098-2098-2098  |  Adversary: 0.0948 [ 22/232]
233 Gold-Original-Attribute: 1485-1485-1485  |  Adversary: 0.0944 [ 22/233]
234 Gold-Original-Attribute: 1796-1796-1796  |  Adversary: 0.0940 [ 22/234]
235 Gold-Original-Attribute:  966- 966- 966  |  Adversary: 0.0936 [ 22/235]
236 Gold-Original-Attribute:  656- 656- 656  |  Adversary: 0.0932 [ 22/236]
237 Gold-Ori